In [ ]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

parameters["form_compiler"]["optimize"]     = True
parameters["form_compiler"]["cpp_optimize"] = True

# Problem data

Most of this is ignored for now. Also, initial values are / can be computed with a linear approximation of the problem.

In [ ]:
class ExactInPlaneDisplacements(Expression):     
    """ data for the exact solutions with f,g as given in p.14 """
    def eval(self, values, x):
        values[0] = 0.0
        values[1] = -x[0]*x[1]/4.0
    def value_shape(self):
        return (2,)

class ExactOutOfPlaneDisplacements(Expression):
    """ data for the exact solutions with f,g as given in p.14 """
    def eval(self, values, x):
        # values[0] is for Q.sub(0) and values[1:] for Q.sub(1) when interpolating
        #values[0] = 0.5* x[0]**2 * np.sin(x[1])
        values[0] = x[0] * np.sin(x[1])
        values[1] = - 0.5* x[0]**2 * np.cos(x[1])
    def value_shape(self):
        return (2,)

In [ ]:
class InitialInPlaneDisplacements(Expression):     
    def eval(self, values, x):
        values[0] = 0.0
        values[1] = -x[1]/10.0
    def value_shape(self):
        return (2,)

class InitialOutOfPlaneDisplacements(Expression):
    def eval(self, values, x):
        #values[0] = 0.5*x[0]**2*(1-x[0])**2*sin(4*DOLFIN_PI*x[1])
        values[0] = x[0]*(1-x[0])*(1-2*x[0])*sin(4*DOLFIN_PI*x[1])
        values[1] = 2*DOLFIN_PI*x[0]**2*(1-x[0])**2*cos(4*DOLFIN_PI*x[1])
    def value_shape(self):
        return (2,)

In [ ]:
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[0]) < DOLFIN_EPS and on_boundary

# Discrete spaces

In-plane displacements (IPD) are discretized with $(P_2)^2$. Out-of-plane displacements with a mixed function space $P_2 \ast (P_1)^2$.

In [ ]:
msh = RectangleMesh(Point(0.0, -0.5), Point(1.0, 0.5), 20, 20)

V = VectorFunctionSpace(msh, "Lagrange", 1, dim=2)        # in plane displacements (IPD)
DE = FiniteElement("Lagrange", msh.ufl_cell(), 2)         # out of plane displacements (OPD)
GE = VectorElement("Lagrange", msh.ufl_cell(), 1, dim=2)  # Gradients of OPD
#W = FunctionSpace(msh, DE*GE)                             # mixed space for OPD
W = FunctionSpace(msh, GE)                             # mixed space for OPD

In [ ]:
uk  = Function(V)    # current solution for in-plane
u_  = Function(V)    # solution from previous step

u_init = InitialInPlaneDisplacements(degree=1)
uk.interpolate(u_init)
u_.interpolate(u_init)

In [ ]:
tau_vals = [1.0]         # time step sizes
gamma = 1.0

def eps(u):
    return grad(u) + grad(u).T

f = Constant(0.0) #Constant(1e-5)
#tau = Constant(tau_vals[-1])

# Boundary conditions

I guess this needs some thinking...

In [ ]:
bcV = DirichletBC(V, Expression(("0.0", "-x[1]/10"), element=V.ufl_element()), DirichletBoundary())
#bcW = DirichletBC(W.sub(0), Constant(0.0), DirichletBoundary())
#bcDW = DirichletBC(W.sub(1), Constant((0.0, 0.0)), DirichletBoundary())
bcDW = DirichletBC(W, Constant((0.0, 0.0)), DirichletBoundary())

# Initial solution / guess

(Maybe) solve a linear problem to obtain some initial `w_, dw_`.

In [ ]:
#wdwk = Function(W)   # current solution for out of plane
#wdw_ = Function(W)   # solution from previous step

dwk = Function(W)   # current solution for out of plane
dw_ = Function(W)   # solution from previous step


#wk, dwk = split(wdwk)    # displacement and gradient of current solution
#w_, dw_ = split(wdw_)    # displacement and gradient of previous solution

# FIXME: solve linearised problem to compute these?
wdw_init = InitialOutOfPlaneDisplacements(degree=1)
#wdwk.interpolate(wdw_init)
#wdw_.interpolate(wdw_init)
dwk.interpolate(wdw_init)
dw_.interpolate(wdw_init)

wk, dwk = TrialFunctions(W)
p, q = TestFunctions(W)

a = (1 + gamma**2*tau_vals[k])*inner(grad(dwk), grad(q))*dx \
    + 2*tau_vals[k]*inner(dot(eps(u_), dwk), q)*dx \
    + inner(dwk - grad(wk), q)*dx
F = inner(f, p)*dx

wdw_ = Function(W)       # solution from previous step in main loop
solve(a == F, wdw_, [bcW, bcDW])

wdwk = Function(W)       # current solution for out of plane
wk, dwk = split(wdwk)    # displacement and gradient of current solution
w_, dw_ = split(wdw_)    # displacement and gradient of previous solution

In [ ]:
pl.figure(figsize=(10,10))
#_ = plot(w_, title="Initial OPD")
_ = plot(dw_, title="Initial OPD")

# Nonlinear problem

In [ ]:
threshold_iter_newton = 5           # Newton should converge in at most these steps
max_attempts_nonlinear = 10         # Number of times we try to decrease tau before giving up with Newton
max_step_size = 10**5               # max value of tau_vals
e_newton = 1.0e-5                   # tolerance for Newton
e_stop = msh.hmin()*1e-8

def setup_newton(W, dwk, dw_, bcs, tau, omega=1.0, mu=1e5):
    #p, q = TestFunctions(W)
    q = TestFunction(W)
    #wk, dwk = split(wdwk)    # displacement and gradient of current solution
    #w_, dw_ = split(wdw_)    # displacement and gradient of previous solution
    F = (1+gamma**2*tau) * inner(grad(dwk), grad(q))*dx - inner(grad(dw_), grad(q))*dx \
        + 2*tau*inner(inner(dwk, dwk)*dwk + dot(eps(u_), (dwk + dw_)/2), q)*dx \
        + mu*tau*curl(dwk)*curl(q)*dx
        #+ tau*inner(f, p)*dx \
        #+ 1e5*inner(dwk - grad(wk), q)*dx
        #+ inner(dwk - grad(wk), dwk - grad(wk)) * p * dx  # FIXME: this just wrong...
    # FIXME: this should also have a different test function as in L0, right?

    # FIXME! I need two different test functions here
    #F1 = inner(wk, grad(p)[0])*dx + inner(dwk[0],p)*dx \
    #     + inner(wk, grad(p)[1])*dx + inner(dwk[1],p)*dx
    #F = F0 + F1
    
    dpq = TrialFunction(W)  # direction along which to compute the derivative of the functional
    J = derivative(F, dwk, dpq)
    problem = NonlinearVariationalProblem(F, dwk, bcs, J)
    solver = NonlinearVariationalSolver(problem)
    prm = solver.parameters
    prm['newton_solver']['absolute_tolerance'] = e_newton/10.0
    prm['newton_solver']['relative_tolerance'] = e_newton
    prm['newton_solver']['maximum_iterations'] = 20
    prm['newton_solver']['relaxation_parameter'] = omega
    return solver

#solver = setup_newton(W, wdwk, wdw_, [bcW, bcDW], tau_vals[-1])
solver = setup_newton(W, dwk, dw_, bcDW, tau_vals[-1])
set_log_level(PROGRESS)
file = File("displacements.pvd")

In [ ]:
# Old stuff:

#class OutOfPlane(NonlinearProblem):
#    def __init__(self, a, L):
#        NonlinearProblem.__init__(self)
#        self.L = L
#        self.a = a
#    def F(self, b, x):
#        assemble(self.L, tensor=b)
#    def J(self, A, x):
#        assemble(self.a, tensor=A)

#problem = OutOfPlane(J, F)  # FIXME! what about bcs??
#solver = NewtonSolver()

#solver.parameters["linear_solver"] = "lu" #"mumps"
#solver.parameters["convergence_criterion"] = "incremental"
#solver.parameters["relative_tolerance"] = e_newton

# Linear problem

In [ ]:
u, z = TrialFunction(V), TestFunction(V)
g  = Constant((0, 0))
L3 = inner(eps(u_), eps(z))*dx + tau_vals[-1]*inner(outer(dw_, dw_), eps(z))*dx + tau_vals[-1]*inner(g,z)*dx
b = inner((1+tau_vals[-1])*eps(u), eps(z))*dx

# Main loop

Iteratively solve the nonlinear and the linear problem until convergence. At each step the former is run with decreasingly small $\tau_k$ until Newton converges in under `threshold_iter_newton` (e.g. 5) iterations.

In [ ]:
k = 0 # current iteration
omega = 1.0  # relaxation parameter
tau_vals = [1.0]
curls = []
dwk.interpolate(wdw_init)
dw_.interpolate(wdw_init)
while True:
    print("Step %d: tau_vals = %.3f" % (k, tau_vals[-1]))
    curls.append(assemble(curl(dwk)*dx))
    print("Current curl = %e" % curls[-1])
    # solve OOP until newton ends in less than threshold_iter_newton steps
    j = 0
    while j < max_attempts_nonlinear:
        #solver = setup_newton(W, wdwk, wdw_, [bcW, bcDW], tau_vals[-1], omega)
        solver = setup_newton(W, dwk, dw_, bcDW, tau_vals[-1], omega, 1e8)
        print("    Running Newton with tau = %.3f, omega = %.2f" % (tau_vals[-1], omega))
        #wdw_.vector()[:] = wdwk.vector()
        dw_.vector()[:] = dwk.vector()
        try:
            niter, converged = solver.solve()
            if niter < threshold_iter_newton and converged:
                print("    Newton converged in %d iterations" % niter)
                break
        except RuntimeError as e:
            print("    Newton did not converge")  # FIXME: technically we are not sure this is the problem
            omega *= 0.5
        tau_vals[-1] *= 0.7
        #replace(F, {tau: Constant(tau_vals[-1])})
        j += 1

    u_.vector()[:] = uk.vector()
    L3 = inner(eps(u_), eps(z))*dx + tau_vals[-1]*inner(outer(dw_, dw_), eps(z))*dx + tau_vals[-1]*inner(g,z)*dx
    b = inner((1+tau_vals[-1])*eps(u), eps(z))*dx
    solve(b == L3, uk, bcV)
    test1 = assemble(inner(grad(dwk-dw_), grad(dwk-dw_))*dx)
    test2 = assemble(inner(eps(uk-u_), eps(uk-u_))*dx)
    if test1+test2 <= tau_vals[-1]*e_stop*min(1.0, tau_vals[-1]):
        print("Done with update of norm %.3f" % (test1+test2))
        break
    file << (uk, float(k))
    #file << (wdwk.split()[0], tau_vals[-1])
    file << (dwk, float(k))
    tau_vals.append(min(2*tau_vals[-1], max_step_size))
    # FIXME: replace is NOT WORKING! 
    #replace(F, {tau: Constant(tau_vals[-1])})
    #replace(L3, {tau: Constant(tau_vals[-1])})
    #replace(b, {tau: Constant(tau_vals[-1])})
    k += 1

In [ ]:
pl.plot(tau_vals)

In [ ]:
orig = Function(W)
orig.interpolate(wdw_init)

diff = project(orig - dwk)
print(norm(diff.vector(), 'l2'))
print(norm(diff.vector(), 'linf'))

In [ ]:
pl.figure(figsize=(22,8))
pl.subplot(1,3,1)
_ = plot(orig, title="Init: %e - %e" % (orig.vector().min(), orig.vector().max()))
pl.subplot(1,3,2)
_ = plot(dwk, title="Sol: %e - %e" % (dwk.vector().min(), dwk.vector().max()))
pl.subplot(1,3,3)
_ = plot(diff, title="Diff: %e - %e" % (diff.vector().min(), diff.vector().max()))